# Результаты CSTR, обученного на MJSynth + SynthText

In [1]:
import sys  
sys.path.insert(0, '/home/astrashnov/vedastr/tools/')

import pandas as pd
from tqdm.notebook import tqdm
from eval_predictions import general_stat

%cd /home/astrashnov/vedastr/
!vedastr_root=${PWD}

/home/astrashnov/vedastr


In [2]:
imgs_dirs = ('ic03_860/images', 'ic13/images', 'ic15/ch4_test_word_images_gt', 'iiit5k/test', 'svt/par', 'svt-perspective/par1')
gt_paths = ('ic03_860/gt.txt', 'ic13/gt.txt', 'ic15/test_groundtruth_1811.txt',
            'iiit5k/iiit5k_1k_test.txt', 'svt/gt.txt', 'svt-perspective/gt.txt')
names = ('ICDAR 2003', 'ICDAR 2013', 'ICDAR 2015', 'IIIT5K', 'SVT', 'SVTP')
sizes = (860, 857, 1811, 3000, 647, 645)

alphabet = "0123456789abcdefghijklmnopqrstuvwxyz"

In [3]:
def get_results(config, checkpoint, predicts):
    results = {}
    for name, img_dir, gt_path in zip(names, imgs_dirs, gt_paths):
        print(name)
        %env CUDA_VISIBLE_DEVICES=3
        !python tools/inference.py $config $checkpoint /home/astrashnov/data/data_clean/$img_dir $predicts > /dev/null
        stat = general_stat(predicts, '/home/astrashnov/data/data_clean/' + gt_path, alphabet=alphabet)
        results[name] = stat
        print()
    df = pd.DataFrame.from_dict(results).transpose()
    acc_columns = ['Accuracy (MC, all words)',
                   'Accuracy (MC, long words)',
                   'Accuracy (MC, short words)',]
    all_columns = df.columns
    for column in acc_columns:
        df[column] *= 100
    df = df.transpose()
    df['All'] = 0.0
    for name, size in zip(names, sizes):
        df['All'] += df[name] * size
    df['All'] /= sum(sizes)
    return df.style.format('{:.4f}').format('{:.2f}%', subset=pd.IndexSlice[acc_columns, :])

#### Обученная на датасете с аугументациями:

In [4]:
get_results('configs/cstr_MJSST_MC_Num.py',
            './workdir/cstr_MJSST_MC_Num/best_norm.pth',
            'workdir/predicts_cstr_MJSST_MC_Num.txt')

ICDAR 2003
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

ICDAR 2013
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

ICDAR 2015
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

IIIT5K
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/

,ICDAR 2003,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,All
"Accuracy (MC, all words)",95.70%,94.63%,81.12%,86.53%,87.64%,82.02%,86.89%
"Accuracy (MC, long words)",90.62%,92.47%,76.56%,68.59%,93.02%,92.68%,79.49%
"Accuracy (MC, short words)",96.11%,94.90%,81.28%,87.52%,87.25%,81.29%,87.29%
"Normalized edit distance 1 (MC, all words)",0.0171,0.0178,0.0805,0.0551,0.0417,0.0785,0.0535
"Normalized edit distance 1 (MC, long words)",0.0207,0.0177,0.0600,0.2164,0.0108,0.0293,0.1044
"Normalized edit distance 1 (MC, short words)",0.0168,0.0178,0.0813,0.0462,0.0439,0.0818,0.0507
"Normalized edit distance 2 (MC, all words)",0.0183,0.0158,0.0727,0.0660,0.0374,0.0763,0.0553
"Normalized edit distance 2 (MC, long words)",0.0207,0.0174,0.0612,0.2562,0.0112,0.0283,0.1199
"Normalized edit distance 2 (MC, short words)",0.0179,0.0153,0.0735,0.0397,0.0409,0.0824,0.0461


#### Предобученная модель из репозитория:

In [5]:
get_results('configs/cstr_MJSST_MC_Num.py',
            './pretrain/cstr.pth',
            'workdir/predicts_cstr_pretrained.txt')

ICDAR 2003
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

ICDAR 2013
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

ICDAR 2015
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

IIIT5K
env: CUDA_VISIBLE_DEVICES=3
/home/astrashnov/anaconda3/envs/vedastr/lib/python3.6/site-packages/torch/

,ICDAR 2003,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,All
"Accuracy (MC, all words)",94.88%,96.62%,85.64%,93.73%,90.11%,84.96%,91.28%
"Accuracy (MC, long words)",93.75%,95.70%,79.69%,73.08%,90.70%,92.68%,82.44%
"Accuracy (MC, short words)",94.97%,96.73%,85.86%,94.87%,90.07%,84.44%,91.74%
"Normalized edit distance 1 (MC, all words)",0.0236,0.0117,0.0644,0.0245,0.0358,0.0712,0.0370
"Normalized edit distance 1 (MC, long words)",0.0163,0.0158,0.0396,0.1035,0.0152,0.0268,0.0559
"Normalized edit distance 1 (MC, short words)",0.0242,0.0112,0.0653,0.0202,0.0372,0.0742,0.0360
"Normalized edit distance 2 (MC, all words)",0.0228,0.0113,0.0567,0.0321,0.0327,0.0702,0.0377
"Normalized edit distance 2 (MC, long words)",0.0163,0.0145,0.0408,0.1389,0.0157,0.0259,0.0696
"Normalized edit distance 2 (MC, short words)",0.0239,0.0105,0.0579,0.0174,0.0349,0.0758,0.0330
